# FB2NEP Workbook 5 – Data Collection and Cleaning

This workbook introduces:

- Data collection pipelines in nutritional epidemiology.
- Identification of implausible or inconsistent values.
- Variable types (continuous, ordinal, categorical, count).
- Handling missing data (MCAR, MAR, MNAR – introduction).
- Simple validation and visual checks.

The code cells are deliberately verbose and heavily commented.

In [ ]:
# 1. Setup: loading Python packages and the dataset

from __future__ import annotations

import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Define a relative path to the data file.
DATA_PATH = pathlib.Path("data") / "fb2nep_synthetic.csv"

# Read the dataset.
df = pd.read_csv(DATA_PATH)

# Inspect the first few rows.
df.head()

## 2. Data collection pipelines

In nutritional epidemiology, data often come from several sources:

- Surveys (questionnaires, 24‑hour recalls, FFQs).
- Laboratory measurements (biomarkers, genotyping, metabolomics).
- Registers and administrative data (hospital admissions, mortality, cancer registries).

These sources are linked through identifiers (for example, participant ID).

In [ ]:
# Simple illustration of separate tables that would be merged in a pipeline

participants = pd.DataFrame({
    "id": [1, 2, 3],
    "age": [52, 47, 19],
    "sex": ["F", "M", "F"],
})

diet_survey = pd.DataFrame({
    "id": [1, 1, 2, 3, 3],
    "day": [1, 2, 1, 1, 2],
    "energy_kcal": [2100, 1900, 3200, 1500, 1600],
})

biomarkers = pd.DataFrame({
    "id": [1, 2, 3],
    "cholesterol_mmol_l": [5.1, 6.8, 4.2],
})

# Merge survey and participant data
diet_with_participants = diet_survey.merge(participants, on="id", how="left")
diet_with_participants

## 3. Variable types

Common variable types in epidemiology:

- **Continuous**: many numerical values on a scale (for example, BMI).
- **Ordinal**: ordered categories (for example, self‑rated health).
- **Categorical (nominal)**: unordered categories (for example, sex).
- **Count**: non‑negative integers (for example, number of GP visits).

The data type in `pandas` affects how variables are summarised and plotted.

In [ ]:
# Inspect variable types in the main dataset

df.dtypes

In [ ]:
# Example: convert smoking code to a categorical variable, if present

if "smoking" in df.columns:
    smoking_map = {0: "never", 1: "former", 2: "current"}
    df["smoking_cat"] = df["smoking"].map(smoking_map).astype("category")
    df[["smoking", "smoking_cat"]].head()

## 4. Identifying implausible or inconsistent values

Implausible values can arise from data entry errors, unit confusion, or misunderstandings.
A first step is simple range checks and summary statistics.

In [ ]:
# Example: check BMI distribution and flag implausible values

if "bmi" in df.columns:
    display(df["bmi"].describe())

    implausible_bmi = df[(df["bmi"] < 10) | (df["bmi"] > 70)]
    implausible_bmi.head()

In [ ]:
# Histogram of BMI to visualise possible outliers

if "bmi" in df.columns:
    plt.figure(figsize=(6, 4))
    df["bmi"].hist(bins=30)
    plt.xlabel("BMI (kg/m²)")
    plt.ylabel("Number of participants")
    plt.title("Distribution of BMI – initial check")
    plt.tight_layout()
    plt.show()

## 5. Missing data: MCAR, MAR, MNAR

We distinguish three broad mechanisms:

- **MCAR** – Missing Completely At Random.
- **MAR** – Missing At Random (depends only on observed variables).
- **MNAR** – Missing Not At Random (depends on unobserved values).

We cannot prove which mechanism holds, but we can explore patterns.

In [ ]:
# Proportion of missing values in each variable

missing_fraction = df.isna().mean().sort_values(ascending=False)
missing_fraction

In [ ]:
# Bar plot of missingness

plt.figure(figsize=(8, 4))
missing_fraction.plot(kind="bar")
plt.ylabel("Proportion missing")
plt.title("Proportion of missing values by variable")
plt.tight_layout()
plt.show()

In [ ]:
# Compare mean age for participants with and without missing BMI

if {"bmi", "age"}.issubset(df.columns):
    df["bmi_missing"] = df["bmi"].isna()
    df.groupby("bmi_missing")["age"].mean()

## 6. Simple validation and visual checks

Further useful checks:

- Numbers of unique values in categorical variables.
- Cross‑tabulations to inspect consistency.
- Plots of key continuous variables against each other.

In [ ]:
# Unique values in a categorical variable

if "smoking_cat" in df.columns:
    df["smoking_cat"].value_counts(dropna=False)

In [ ]:
# Cross‑tabulation of sex and smoking category

if {"sex", "smoking_cat"}.issubset(df.columns):
    pd.crosstab(df["sex"], df["smoking_cat"], margins=True)

In [ ]:
# Scatter plot of BMI vs age

if {"bmi", "age"}.issubset(df.columns):
    plt.figure(figsize=(6, 4))
    plt.scatter(df["age"], df["bmi"], alpha=0.4)
    plt.xlabel("Age (years)")
    plt.ylabel("BMI (kg/m²)")
    plt.title("BMI vs age – simple data check")
    plt.tight_layout()
    plt.show()